In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,18678.94,0.11,0.15,-0.04,0.04,0.02,-0.00,-0.00
3,0.11,5005.83,-0.12,-0.02,0.01,-0.00,0.00,-0.00
4,0.15,-0.12,1581.11,-0.01,0.01,0.02,0.00,0.00
5,-0.04,-0.02,-0.01,447.93,-0.01,-0.01,-0.00,-0.00
6,0.04,0.01,0.01,-0.01,107.28,0.00,0.00,0.00
7,0.02,-0.00,0.02,-0.01,0.00,23.56,-0.00,0.00
8,-0.00,0.00,0.00,-0.00,0.00,-0.00,5.01,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.93


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00003,0.00004,-0.00002,-0.00002
3,0.00001,1.00000,-0.00004,-0.00001,0.00002,-0.00001,0.00002,-0.00006
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00008,0.00000,0.00001
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00003,-0.00006,-0.00001,-0.00001
6,0.00003,0.00002,0.00001,-0.00003,1.00000,0.00002,0.00002,0.00002
7,0.00004,-0.00001,0.00008,-0.00006,0.00002,1.00000,-0.00002,0.00010
8,-0.00002,0.00002,0.00000,-0.00001,0.00002,-0.00002,1.00000,0.00002
9,-0.00002,-0.00006,0.00001,-0.00001,0.00002,0.00010,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.3241318918376112

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.36881757e+07 3.07388309e+06 2.97759736e+05 2.29368186e+04
 1.23534176e+03 5.32196475e+01 1.83066527e+00 1.52235640e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999078,-0.041079,-0.012003,-0.003380,-0.000807,-0.000176,-0.000038,-0.000007
1,0.040249,0.997347,-0.058697,-0.014830,-0.003517,-0.000774,-0.000164,-0.000031
2,0.014094,0.057086,0.996229,-0.062274,-0.013547,-0.002950,-0.000629,-0.000116
3,0.004775,0.017924,0.060243,0.995856,-0.064160,-0.013140,-0.002780,-0.000515
4,0.001418,0.005299,0.016835,0.061908,0.994808,-0.077243,-0.015397,-0.002841
5,0.000410,0.001535,0.004849,0.017198,0.073999,0.990750,-0.110682,-0.019116
6,0.000128,0.000477,0.001510,0.005337,0.022293,0.103808,0.976975,-0.184977
7,0.000049,0.000182,0.000575,0.002032,0.008478,0.038588,0.181728,0.982553


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0009222442459448921,
 0.0026526600568284042,
 0.0037712253834113474,
 0.004143830125094561,
 0.005192405439225056,
 0.009249572436364595,
 0.023025417951014093,
 0.017447423529944217]